In [1]:
from IPython.core.magic import Magics, cell_magic, line_magic, magics_class
from pexpect import spawn

TIMEOUT = 300
PROG = "hive"
PROMPT = ["\r\n    > ", "\r\nhive> "]
QUIT = "quit;"


@magics_class
class Magic(Magics):
    def __init__(self, shell):
        super().__init__(shell)
        self.app = spawn(PROG, timeout=60)
        self.app.expect(PROMPT)

    @cell_magic
    def hive(self, line, cell):
        cell_lines = [cell_line.strip() for cell_line in cell.split("\n")]
        cell_lines = [cell_line for cell_line in cell_lines if cell_line != ""]
        for cell_line in cell_lines:
            self.app.sendline(cell_line)
            self.app.expect(PROMPT, timeout=TIMEOUT)
            output = self.app.before.decode()
            output = output.replace("\r\n", "\n")
            output = output.split("\n")
            output = [output_line.strip() for output_line in output]
            for output_line in output:
                if output_line not in cell_lines:
                    print(output_line)
        return None

    @line_magic
    def quit(self, line):
        self.app.sendline(QUIT)


def load_ipython_extension(ip):
    ip.register_magics(Magic(ip))


load_ipython_extension(ip=get_ipython())

In [ ]:
!hdfs dfs -copyFromLocal pregunta_02/data.tsv /tmp

In [ ]:
!hdfs dfs -ls /tmp

In [ ]:
!hdfs dfs -rm /tmp/data.tsv

In [ ]:
%%hive
DROP TABLE IF EXISTS data;

In [ ]:
%%hive
CREATE TABLE data (
letter STRING,
date_event STRING,
value INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY  '\t';

In [ ]:
%%hive
LOAD DATA INPATH '/tmp/data.tsv' OVERWRITE INTO TABLE data;
SELECT * FROM data;

In [ ]:
%%hive
SELECT DISTINCT(value)
FROM data
ORDER BY value LIMIT 5;

In [ ]:
%%hive
SELECT DISTINCT(letras) as let
FROM(
SELECT
explode(c5) as letras
FROM
tbl0
) w;
ORDER BY let

In [2]:
!hdfs dfs -copyFromLocal pregunta_04/data0.csv /tmp

In [3]:
!hdfs dfs -copyFromLocal pregunta_04/data1.csv /tmp

In [4]:
!hdfs dfs -ls /tmp

Found 4 items
-rw-r--r--   1 root supergroup        397 2023-06-03 15:23 /tmp/data0.csv
-rw-r--r--   1 root supergroup        282 2023-06-03 15:23 /tmp/data1.csv
drwxrwx---   - root supergroup          0 2023-06-03 15:20 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2023-06-03 15:22 /tmp/hive


In [5]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA INPATH '/tmp/data0.csv' INTO TABLE tbl0;

OK
Time taken: 11.383 seconds
OK
Time taken: 1.123 seconds
Loading data to table default.tbl0
OK
Time taken: 1.278 seconds


In [6]:
%%hive
SHOW TABLES;

OK
tbl0
Time taken: 0.255 seconds, Fetched: 1 row(s)


In [58]:
%%hive
SELECT SUBSTRING(c4,0,4) as c4, letras, count(1) FROM tbl0
LATERAL VIEW
    explode(c5) tbl0 as letras
GROUP BY c4, letras;

FAILED: SemanticException 4:9 SELECT DISTINCT and GROUP BY can not be in the same query. Error encountered near token 'letras'


In [56]:
%%hive
SELECT DISTINCT(SUBSTRING(c4,0,4)) as c4 FROM tbl0;

Query ID = root_20230603161039_e51c8d54-5dec-4db9-b4f7-7f998bc5e624
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
set mapreduce.job.reduces=<number>
Starting Job = job_1685805632661_0018, Tracking URL = http://6c0dee391b46:8088/proxy/application_1685805632661_0018/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1685805632661_0018
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2023-06-03 16:10:47,160 Stage-1 map = 0%,  reduce = 0%
2023-06-03 16:10:52,328 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 2.93 sec
2023-06-03 16:10:57,481 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU 5.3 sec
MapReduce Total cumulative CPU time: 5 seconds 300 

In [ ]:
%%hive
SELECT explode(split(c5, ',')) AS word FROM tbl0 LIMIT 5;

In [68]:
%%hive
SELECT otro, letras, count(*) as total FROM(
SELECT SUBSTRING(c4,0,4) as otro, letras FROM tbl0
LATERAL VIEW
    explode(c5) tbl0 as letras
ORDER BY otro, letras
) w;
GROUP BY otro, letras;

NoViableAltException(135@[])
at org.apache.hadoop.hive.ql.parse.HiveParser.statement(HiveParser.java:1300)
at org.apache.hadoop.hive.ql.parse.ParseDriver.parse(ParseDriver.java:208)
at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:77)
at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:70)
at org.apache.hadoop.hive.ql.Driver.compile(Driver.java:468)
at org.apache.hadoop.hive.ql.Driver.compileInternal(Driver.java:1317)
at org.apache.hadoop.hive.ql.Driver.runInternal(Driver.java:1457)
at org.apache.hadoop.hive.ql.Driver.run(Driver.java:1237)
at org.apache.hadoop.hive.ql.Driver.run(Driver.java:1227)
at org.apache.hadoop.hive.cli.CliDriver.processLocalCmd(CliDriver.java:233)
at org.apache.hadoop.hive.cli.CliDriver.processCmd(CliDriver.java:184)
at org.apache.hadoop.hive.cli.CliDriver.processLine(CliDriver.java:403)
at org.apache.hadoop.hive.cli.CliDriver.executeDriver(CliDriver.java:821)
at org.apache.hadoop.hive.cli.CliDriver.run(CliDriver.java:759)
at o

In [70]:
%%hive
SELECT otro, letras, count(*) as total FROM(
SELECT SUBSTRING(c4,0,4) as otro, letras FROM tbl0
LATERAL VIEW
    explode(c5) tbl0 as letras
ORDER BY otro, letras
) w
GROUP BY otro, letras;

Query ID = root_20230603161921_1c0847db-9b84-4b02-bbb8-0494f43db67b
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
set mapreduce.job.reduces=<number>
Starting Job = job_1685805632661_0020, Tracking URL = http://6c0dee391b46:8088/proxy/application_1685805632661_0020/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1685805632661_0020
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2023-06-03 16:19:29,472 Stage-1 map = 0%,  reduce = 0%
2023-06-03 16:19:34,661 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.18 sec
2023-06-03 16:19:40,864 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU 5.55 sec
MapReduce Total cumulative CPU time: 5 seconds 550 msec
Ended Job = j